# Setting up the libraries

In [ ]:
Pkg.add("LinearMaps")
Pkg.clone("https://github.com/Jutho/TensorOperations.jl")
Pkg.update()

# Exact diagonalization of the Ising model

In [5]:
using LinearMaps

In [10]:
# Ising model with transverse magnetic field h (critical h=1 by default)
function build_ising_ham(h=1.0)
    X = [0 1; 1 0]
    Z = [1 0; 0 -1]
    I = eye(2)
    XX = kron(X,X)
    ZI = kron(Z,I)
    IZ = kron(I,Z)
    H = -(XX + h/2*(ZI+IZ))
    return H
end


function apply_spin_hamiltonian{T}(v::AbstractVector{T}, ham::Matrix{T})
    D = length(v)
    N = convert(Int64, log2(D))
    tensor_shp = tuple(fill(2, N)...)  # tensor_shp =(2,2,2, ..., 2)
    cyclperm = tuple(2:N..., 1)  #tuple for cyclic permutation cyclperm = (2, 3, ... N, 1)
    out = zeros(tensor_shp)
    for n=1:N
        v, out = reshape(v, (4, 2^(N-2))), reshape(out, (4, 2^(N-2)))
        out += ham*v
        v, out = reshape(v, tensor_shp), reshape(out, tensor_shp)
        v, out = permutedims(v, cyclperm), permutedims(out, cyclperm) 
    end
    out = reshape(out, D)
    return out
end


N = 14
ising_h = build_ising_ham()
apply_ising_hamiltonian(v) = apply_spin_hamiltonian(v, ising_h)
H = LinearMap(apply_ising_hamiltonian, 2^N)
@time u, v = eigs(H; nev=1, which=:SR)  #:SR stands for smallest real part
energy = real(u[1])
e_density = energy/N
infN_e_density = -4/pi
info(e_density)
info((e_density - infN_e_density)/infN_e_density)

 64.718491 seconds (606.81 M allocations: 29.739 GiB, 5.72% gc time)


INFO: -1.2759148957686433
INFO: 0.002101215788139041


# Uniform Matrix Product States

In [3]:
include("MPSTools.jl")
using MPSTools

In [4]:
d = 2
D = 5
rumps = rand_UMPS(Complex128, d, D)

info(bond_dim(rumps))
info(phys_dim(rumps))
info(mps_tensor(rumps))

INFO: 5
INFO: 2
INFO: Complex{Float64}[0.744652-1.12976im -0.968165-0.224989im; -1.22516-0.957561im 0.316324+0.637425im; -0.110278+0.552591im 0.43988+0.18355im; 0.956878+0.393327im 1.4291-0.0377493im; 0.142396-0.110386im 0.956832-0.911607im]

Complex{Float64}[1.54723+0.701967im -0.310647-0.71533im; -0.381559+0.255508im -0.132735+1.08072im; -0.370723-0.133146im 0.271987-0.238011im; -0.399657+1.31718im 0.632122+0.182089im; 0.275616+0.35622im -0.640325+1.31677im]

Complex{Float64}[-0.509559-0.258974im 0.57498+0.685549im; -1.0083+0.047313im 0.783889-0.0143781im; 0.0950174+1.32624im 0.306357-0.883913im; -0.251599+0.261489im 1.4577-0.792693im; -0.915025+0.00288091im -0.146286-0.369911im]

Complex{Float64}[0.540806+0.734083im 0.426209+0.286974im; 0.286103-0.262267im 0.497595-0.853037im; -0.156529+0.214838im -0.0508923-0.163969im; 0.787602+0.151493im -0.116621+0.941736im; -0.85885-0.0187503im -0.0986024+0.328805im]

Complex{Float64}[-0.663715-0.979546im 2.13778-0.0764858im; 0.750549+0.150234im

# How to use TensorOperations
https://github.com/Jutho/TensorOperations.jl

In [17]:
using TensorOperations

In [19]:
A=randn(5,2,4,5,4,5)
B=randn(5,5,2)
@tensor D[a,b,c] := A[a,e,f,c,f,g]*B[g,b,e]

5×5×5 Array{Float64,3}:
[:, :, 1] =
 -2.12169   -2.99067   -3.74255  -2.59708   5.25028
 -2.50781   -5.4607    -3.37948  -6.93873   2.58659
  4.50568    0.387593   7.47029   2.52363  10.1505 
 -0.988978   0.247945  -5.54016  -1.1716   -7.00871
 -5.94354    7.58316    2.56992  -9.00105   6.45988

[:, :, 2] =
  0.198349  -4.46213    6.3206   -2.8836     1.74121
  5.27555    1.78594    3.38578  -6.23562    2.07318
 -3.51399    0.929723  -9.07545   2.97692    3.71744
  0.635069   1.67097   -6.58773  -4.70528  -14.7961 
 -5.85589   -4.83646   -2.80677  -5.29183    6.72133

[:, :, 3] =
 2.22512   -0.987532   1.58404  -2.78058  -0.811776
 1.47323    5.61518   -2.07568   2.90729  -0.723866
 4.0569     0.984389  11.247     8.49051   4.42938 
 3.66296   -0.70147   11.657    -5.74762  -0.406107
 0.527249  -2.81005   -1.39472  -7.85853   2.43063 

[:, :, 4] =
 -3.07765    -5.19345  -8.07212   -2.85259   0.0464959
 -1.17632    -3.86023   0.991683  -4.35004  -4.96881  
  0.635692    2.96081  -1.9728